In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, roc_auc_score, confusion_matrix, classification_report

# Load Dataset

In [2]:
X_train_resampled = pd.read_csv('../data/X_train_resampled.csv')
y_train_resampled = pd.read_csv('../data/y_train_resampled.csv')
X_train = pd.read_csv('../data/X_train.csv')
y_train = pd.read_csv('../data/y_train.csv')
X_val = pd.read_csv('../data/X_val.csv')
y_val = pd.read_csv('../data/y_val.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv')

# Use Log-Transformed Features

In [3]:
X_train_resampled.columns

Index(['income', 'name_email_similarity', 'prev_address_months_count',
       'current_address_months_count', 'days_since_request', 'zip_count_4w',
       'velocity_6h', 'velocity_24h', 'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'credit_risk_score',
       'email_is_free', 'phone_home_valid', 'phone_mobile_valid',
       'bank_months_count', 'has_other_cards', 'proposed_credit_limit',
       'foreign_request', 'keep_alive_session', 'device_distinct_emails_8w',
       'month', 'age_10', 'age_20', 'age_30', 'age_40', 'age_50', 'age_60',
       'age_70', 'age_80', 'age_90', 'payment_AA', 'payment_AB', 'payment_AC',
       'payment_AD', 'payment_AE', 'employment_CA', 'employment_CB',
       'employment_CC', 'employment_CD', 'employment_CE', 'employment_CF',
       'employment_CG', 'housing_BA', 'housing_BB', 'housing_BC', 'housing_BD',
       'housing_BE', 'housing_BF', 'housing_BG', 'source_INTERNET',
       'source_TELEAPP', 'device_linux', 'device_

In [4]:
log_transformed_columns = ['days_since_request_log', 'intended_balcon_amount_log',
                           'zip_count_4w_log', 'velocity_24h_log', 'velocity_4w_log',
                           'date_of_birth_distinct_emails_4w_log','session_length_in_minutes_log']
original_columns = ['days_since_request', 'intended_balcon_amount_clean',
                    'zip_count_4w', 'velocity_24h', 'velocity_4w',
                    'date_of_birth_distinct_emails_4w', 'session_length_in_minutes_cleaned']

In [5]:
X_train_resampled_log = X_train_resampled.drop(columns=original_columns)
X_train_log = X_train.drop(columns=original_columns)
X_val_log = X_val.drop(columns=original_columns)
X_test_log = X_test.drop(columns=original_columns)

## Use SMOTE Resampled Training Data

### Baseline

In [6]:
# Initialize Logistic Regression model
logreg = LogisticRegression(
    max_iter=5000,
    random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_resampled_log_scaled = scaler.fit_transform(X_train_resampled_log)
X_val_log_scaled = scaler.transform(X_val_log)
X_test_log_scaled = scaler.transform(X_test_log)

In [7]:
# Train (fit) the model
logreg.fit(X_train_resampled_log_scaled, y_train_resampled.values.ravel())

# Evaluate on training data
y_train_pred = logreg.predict(X_train_resampled_log_scaled)
y_train_proba = logreg.predict_proba(X_train_resampled_log_scaled)[:, 1]
print("Training Classification Report:\n")
print(classification_report(y_train_resampled, y_train_pred, digits=4))
roc_auc_train = roc_auc_score(y_train_resampled, y_train_proba)
pr_auc_train = average_precision_score(y_train_resampled, y_train_proba)
print(f"Training ROC-AUC Score: {roc_auc_train:.4f}")
print(f"Training PR-AUC Score:  {pr_auc_train:.4f}")
cm_train = confusion_matrix(y_train_resampled, y_train_pred)
print("\n Training Confusion Matrix:")
print(cm_train)

Training Classification Report:

              precision    recall  f1-score   support

           0     0.9836    0.9975    0.9905    786838
           1     0.9975    0.9833    0.9904    786838

    accuracy                         0.9904   1573676
   macro avg     0.9905    0.9904    0.9904   1573676
weighted avg     0.9905    0.9904    0.9904   1573676

Training ROC-AUC Score: 0.9983
Training PR-AUC Score:  0.9987

 Training Confusion Matrix:
[[784882   1956]
 [ 13107 773731]]


In [8]:
# Predict on validation data
y_pred = logreg.predict(X_val_log_scaled)
y_proba = logreg.predict_proba(X_val_log_scaled)[:, 1]

# Evaluate the model
print("Classification Report:\n")
print(classification_report(y_val, y_pred, digits=4))

roc_auc = roc_auc_score(y_val, y_proba)
pr_auc = average_precision_score(y_val, y_proba)
cm = confusion_matrix(y_val, y_pred)

print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"PR-AUC Score:  {pr_auc:.4f}")
print("\n Confusion Matrix:")
print(cm)

Classification Report:

              precision    recall  f1-score   support

           0     0.9874    0.9985    0.9929    106718
           1     0.3563    0.0607    0.1037      1450

    accuracy                         0.9859    108168
   macro avg     0.6718    0.5296    0.5483    108168
weighted avg     0.9789    0.9859    0.9810    108168

ROC-AUC Score: 0.8636
PR-AUC Score:  0.1383

 Confusion Matrix:
[[106559    159]
 [  1362     88]]


In [9]:
# Predict on test data
y_pred = logreg.predict(X_test_log_scaled)
y_proba = logreg.predict_proba(X_test_log_scaled)[:, 1]

# Evaluate the model
print("Classification Report:\n")
print(classification_report(y_test, y_pred, digits=4))

roc_auc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)

print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"PR-AUC Score:  {pr_auc:.4f}")
print("\n Confusion Matrix:")
print(cm)

Classification Report:

              precision    recall  f1-score   support

           0     0.9860    0.9990    0.9924     95415
           1     0.4186    0.0504    0.0900      1428

    accuracy                         0.9850     96843
   macro avg     0.7023    0.5247    0.5412     96843
weighted avg     0.9776    0.9850    0.9791     96843

ROC-AUC Score: 0.8676
PR-AUC Score:  0.1621

 Confusion Matrix:
[[95315   100]
 [ 1356    72]]


### Regularization tuning

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(logreg, param_grid, scoring='f1', cv=5)
grid.fit(X_train_resampled_log_scaled, y_train_resampled.values.ravel())

,estimator,LogisticRegre...ndom_state=42)
,param_grid,"{'C': [0.01, 0.1, ...]}"
,scoring,'f1'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [11]:
print("Best C:", grid.best_params_['C'])
print("Best cross-validated F1 score:", grid.best_score_)

Best C: 1
Best cross-validated F1 score: 0.9893804062433562


In [12]:
# Evaluate on Training Data with Best Model
best_logreg = grid.best_estimator_
y_train_pred_best = best_logreg.predict(X_train_resampled_log_scaled)
y_train_proba_best = best_logreg.predict_proba(X_train_resampled_log_scaled)[:, 1]
print("Training Classification Report with Best Model:\n")
print(classification_report(y_train_resampled, y_train_pred_best, digits=4))
roc_auc_train_best = roc_auc_score(y_train_resampled, y_train_proba_best)
pr_auc_train_best = average_precision_score(y_train_resampled, y_train_proba_best)
print(f"Training ROC-AUC Score with Best Model: {roc_auc_train_best:.4f}")
print(f"Training PR-AUC Score with Best Model:  {pr_auc_train_best:.4f}")
cm_train_best = confusion_matrix(y_train_resampled, y_train_pred_best)
print("\n Training Confusion Matrix with Best Model:")
print(cm_train_best)

Training Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9836    0.9975    0.9905    786838
           1     0.9975    0.9833    0.9904    786838

    accuracy                         0.9904   1573676
   macro avg     0.9905    0.9904    0.9904   1573676
weighted avg     0.9905    0.9904    0.9904   1573676

Training ROC-AUC Score with Best Model: 0.9983
Training PR-AUC Score with Best Model:  0.9987

 Training Confusion Matrix with Best Model:
[[784882   1956]
 [ 13107 773731]]


In [13]:
# Evaluate the best model on validation set
y_val_pred_best = best_logreg.predict(X_val_log_scaled)
y_val_proba_best = best_logreg.predict_proba(X_val_log_scaled)[:, 1]
print("Validation Classification Report with Best Model:\n")
print(classification_report(y_val, y_val_pred_best, digits=4))
roc_auc_val_best = roc_auc_score(y_val, y_val_proba_best)
pr_auc_val_best = average_precision_score(y_val, y_val_proba_best)
print(f"Validation ROC-AUC Score with Best Model: {roc_auc_val_best:.4f}")
print(f"Validation PR-AUC Score with Best Model:  {pr_auc_val_best:.4f}")
cm_val_best = confusion_matrix(y_val, y_val_pred_best)
print("\n Validation Confusion Matrix with Best Model:")
print(cm_val_best)

Validation Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9874    0.9985    0.9929    106718
           1     0.3563    0.0607    0.1037      1450

    accuracy                         0.9859    108168
   macro avg     0.6718    0.5296    0.5483    108168
weighted avg     0.9789    0.9859    0.9810    108168

Validation ROC-AUC Score with Best Model: 0.8636
Validation PR-AUC Score with Best Model:  0.1383

 Validation Confusion Matrix with Best Model:
[[106559    159]
 [  1362     88]]


In [14]:
# Evaluate the best model on test set
y_test_pred_best = best_logreg.predict(X_test_log_scaled)
y_test_proba_best = best_logreg.predict_proba(X_test_log_scaled)[:, 1]
print("Test Classification Report with Best Model:\n")
print(classification_report(y_test, y_test_pred_best, digits=4))
roc_auc_test_best = roc_auc_score(y_test, y_test_proba_best)
pr_auc_test_best = average_precision_score(y_test, y_test_proba_best)
print(f"Test ROC-AUC Score with Best Model: {roc_auc_test_best:.4f}")
print(f"Test PR-AUC Score with Best Model:  {pr_auc_test_best:.4f}")
cm_test_best = confusion_matrix(y_test, y_test_pred_best)
print("\n Test Confusion Matrix with Best Model:")
print(cm_test_best)

Test Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9860    0.9990    0.9924     95415
           1     0.4186    0.0504    0.0900      1428

    accuracy                         0.9850     96843
   macro avg     0.7023    0.5247    0.5412     96843
weighted avg     0.9776    0.9850    0.9791     96843

Test ROC-AUC Score with Best Model: 0.8676
Test PR-AUC Score with Best Model:  0.1621

 Test Confusion Matrix with Best Model:
[[95315   100]
 [ 1356    72]]


## Use original train with class weighting

### Baseline

In [15]:
# Initialize Logistic Regression model
logreg = LogisticRegression(
    max_iter=5000,
    random_state=42,
    class_weight='balanced'
)

# Scale features
scaler = StandardScaler()
X_train_log_scaled = scaler.fit_transform(X_train_log)
X_val_log_scaled = scaler.transform(X_val_log)

In [16]:
# Train (fit) the model
logreg.fit(X_train_log_scaled, y_train.values.ravel())

# Evaluate on training data
y_train_pred = logreg.predict(X_train_log_scaled)
y_train_proba = logreg.predict_proba(X_train_log_scaled)[:, 1]
print("Training Classification Report:\n")
print(classification_report(y_train, y_train_pred, digits=4))
roc_auc_train = roc_auc_score(y_train, y_train_proba)
pr_auc_train = average_precision_score(y_train, y_train_proba)
print(f"Training ROC-AUC Score: {roc_auc_train:.4f}")
print(f"Training PR-AUC Score:  {pr_auc_train:.4f}")
cm_train = confusion_matrix(y_train, y_train_pred)
print("\n Training Confusion Matrix:")
print(cm_train)

Training Classification Report:

              precision    recall  f1-score   support

           0     0.9973    0.8049    0.8909    786838
           1     0.0403    0.7898    0.0766      8151

    accuracy                         0.8048    794989
   macro avg     0.5188    0.7974    0.4837    794989
weighted avg     0.9875    0.8048    0.8825    794989

Training ROC-AUC Score: 0.8764
Training PR-AUC Score:  0.1287

 Training Confusion Matrix:
[[633363 153475]
 [  1713   6438]]


In [17]:
# Predict on validation data
y_pred = logreg.predict(X_val_log_scaled)
y_proba = logreg.predict_proba(X_val_log_scaled)[:, 1]

# Evaluate the model
print("Classification Report:\n")
print(classification_report(y_val, y_pred, digits=4))

roc_auc = roc_auc_score(y_val, y_proba)
pr_auc = average_precision_score(y_val, y_proba)
cm = confusion_matrix(y_val, y_pred)

print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"PR-AUC Score:  {pr_auc:.4f}")
print("\n Confusion Matrix:")
print(cm)

Classification Report:

              precision    recall  f1-score   support

           0     0.9961    0.8277    0.9041    106718
           1     0.0567    0.7628    0.1056      1450

    accuracy                         0.8268    108168
   macro avg     0.5264    0.7952    0.5049    108168
weighted avg     0.9835    0.8268    0.8934    108168

ROC-AUC Score: 0.8772
PR-AUC Score:  0.1560

 Confusion Matrix:
[[88330 18388]
 [  344  1106]]


In [18]:
# Predict on test data
y_pred = logreg.predict(X_test_log_scaled)
y_proba = logreg.predict_proba(X_test_log_scaled)[:, 1]
# Evaluate the model
print("Classification Report:\n")
print(classification_report(y_test, y_pred, digits=4)) 
roc_auc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"PR-AUC Score:  {pr_auc:.4f}")
print("\n Confusion Matrix:")
print(cm)   

Classification Report:

              precision    recall  f1-score   support

           0     0.9902    0.9791    0.9846     95415
           1     0.2023    0.3536    0.2574      1428

    accuracy                         0.9699     96843
   macro avg     0.5963    0.6664    0.6210     96843
weighted avg     0.9786    0.9699    0.9739     96843

ROC-AUC Score: 0.8811
PR-AUC Score:  0.1815

 Confusion Matrix:
[[93424  1991]
 [  923   505]]


### Regularization Tuning

In [19]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(logreg, param_grid, scoring='f1', cv=5)
grid.fit(X_train_log_scaled, y_train.values.ravel())

,estimator,LogisticRegre...ndom_state=42)
,param_grid,"{'C': [0.01, 0.1, ...]}"
,scoring,'f1'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [20]:
print("Best C:", grid.best_params_['C'])
print("Best cross-validated F1 score:", grid.best_score_)

Best C: 0.1
Best cross-validated F1 score: 0.08040153542704467


In [21]:
# Evaluate on Training Data with Best Model
best_logreg = grid.best_estimator_
y_train_pred_best = best_logreg.predict(X_train_log_scaled)
y_train_proba_best = best_logreg.predict_proba(X_train_log_scaled)[:, 1]
print("Training Classification Report with Best Model:\n")
print(classification_report(y_train, y_train_pred_best, digits=4))
roc_auc_train_best = roc_auc_score(y_train, y_train_proba_best)
pr_auc_train_best = average_precision_score(y_train, y_train_proba_best)
print(f"Training ROC-AUC Score with Best Model: {roc_auc_train_best:.4f}")
print(f"Training PR-AUC Score with Best Model:  {pr_auc_train_best:.4f}")
cm_train_best = confusion_matrix(y_train, y_train_pred_best)
print("\n Training Confusion Matrix with Best Model:")
print(cm_train_best)

Training Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9973    0.8049    0.8909    786838
           1     0.0403    0.7898    0.0766      8151

    accuracy                         0.8048    794989
   macro avg     0.5188    0.7974    0.4837    794989
weighted avg     0.9875    0.8048    0.8825    794989

Training ROC-AUC Score with Best Model: 0.8764
Training PR-AUC Score with Best Model:  0.1287

 Training Confusion Matrix with Best Model:
[[633364 153474]
 [  1713   6438]]


In [22]:
# Evaluate the best model on validation set
y_val_pred_best = best_logreg.predict(X_val_log_scaled)
y_val_proba_best = best_logreg.predict_proba(X_val_log_scaled)[:, 1]
print("Validation Classification Report with Best Model:\n")
print(classification_report(y_val, y_val_pred_best, digits=4))
roc_auc_val_best = roc_auc_score(y_val, y_val_proba_best)
pr_auc_val_best = average_precision_score(y_val, y_val_proba_best)
print(f"Validation ROC-AUC Score with Best Model: {roc_auc_val_best:.4f}")
print(f"Validation PR-AUC Score with Best Model:  {pr_auc_val_best:.4f}")
cm_val_best = confusion_matrix(y_val, y_val_pred_best)
print("\n Validation Confusion Matrix with Best Model:")
print(cm_val_best)

Validation Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9961    0.8277    0.9041    106718
           1     0.0567    0.7628    0.1056      1450

    accuracy                         0.8268    108168
   macro avg     0.5264    0.7952    0.5049    108168
weighted avg     0.9835    0.8268    0.8934    108168

Validation ROC-AUC Score with Best Model: 0.8772
Validation PR-AUC Score with Best Model:  0.1560

 Validation Confusion Matrix with Best Model:
[[88330 18388]
 [  344  1106]]


In [23]:
# Evaluate the best model on test set
y_test_pred_best = best_logreg.predict(X_test_log_scaled)
y_test_proba_best = best_logreg.predict_proba(X_test_log_scaled)[:, 1]
print("Test Classification Report with Best Model:\n")
print(classification_report(y_test, y_test_pred_best, digits=4))
roc_auc_test_best = roc_auc_score(y_test, y_test_proba_best)
pr_auc_test_best = average_precision_score(y_test, y_test_proba_best)
print(f"Test ROC-AUC Score with Best Model: {roc_auc_test_best:.4f}")
print(f"Test PR-AUC Score with Best Model:  {pr_auc_test_best:.4f}")
cm_test_best = confusion_matrix(y_test, y_test_pred_best)
print("\n Test Confusion Matrix with Best Model:")
print(cm_test_best)

Test Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9902    0.9791    0.9846     95415
           1     0.2022    0.3536    0.2573      1428

    accuracy                         0.9699     96843
   macro avg     0.5962    0.6664    0.6210     96843
weighted avg     0.9786    0.9699    0.9739     96843

Test ROC-AUC Score with Best Model: 0.8811
Test PR-AUC Score with Best Model:  0.1815

 Test Confusion Matrix with Best Model:
[[93423  1992]
 [  923   505]]


# Use Non Log-Transformed Features

In [24]:
X_train_resampled_nolog = X_train_resampled.drop(columns=log_transformed_columns)
X_train_nolog = X_train.drop(columns=log_transformed_columns)
X_val_nolog = X_val.drop(columns=log_transformed_columns)
X_test_nolog = X_test.drop(columns=log_transformed_columns)

## Use SMOTE Resampled Training Data

### Baseline

In [26]:
# Initialize Logistic Regression model
logreg = LogisticRegression(
    max_iter=5000,
    random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_resampled_nolog_scaled = scaler.fit_transform(X_train_resampled_nolog)
X_val_nolog_scaled = scaler.transform(X_val_nolog)
X_test_nolog_scaled = scaler.transform(X_test_nolog)

In [27]:
# Train (fit) the model
logreg.fit(X_train_resampled_nolog_scaled, y_train_resampled.values.ravel())

# Evaluate on training data
y_train_pred = logreg.predict(X_train_resampled_nolog_scaled)
y_train_proba = logreg.predict_proba(X_train_resampled_nolog_scaled)[:, 1]
print("Training Classification Report:\n")
print(classification_report(y_train_resampled, y_train_pred, digits=4))
roc_auc_train = roc_auc_score(y_train_resampled, y_train_proba)
pr_auc_train = average_precision_score(y_train_resampled, y_train_proba)
print(f"Training ROC-AUC Score: {roc_auc_train:.4f}")
print(f"Training PR-AUC Score:  {pr_auc_train:.4f}")
cm_train = confusion_matrix(y_train_resampled, y_train_pred)
print("\n Training Confusion Matrix:")
print(cm_train)

Training Classification Report:

              precision    recall  f1-score   support

           0     0.9835    0.9975    0.9905    786838
           1     0.9975    0.9833    0.9903    786838

    accuracy                         0.9904   1573676
   macro avg     0.9905    0.9904    0.9904   1573676
weighted avg     0.9905    0.9904    0.9904   1573676

Training ROC-AUC Score: 0.9983
Training PR-AUC Score:  0.9987

 Training Confusion Matrix:
[[784883   1955]
 [ 13158 773680]]


In [28]:
# Predict on validation data
y_pred = logreg.predict(X_val_nolog_scaled)
y_proba = logreg.predict_proba(X_val_nolog_scaled)[:, 1]

# Evaluate the model
print("Classification Report:\n")
print(classification_report(y_val, y_pred, digits=4))

roc_auc = roc_auc_score(y_val, y_proba)
pr_auc = average_precision_score(y_val, y_proba)
cm = confusion_matrix(y_val, y_pred)

print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"PR-AUC Score:  {pr_auc:.4f}")
print("\n Confusion Matrix:")
print(cm)

Classification Report:

              precision    recall  f1-score   support

           0     0.9874    0.9984    0.9929    106718
           1     0.3399    0.0593    0.1010      1450

    accuracy                         0.9858    108168
   macro avg     0.6636    0.5289    0.5469    108168
weighted avg     0.9787    0.9858    0.9809    108168

ROC-AUC Score: 0.8628
PR-AUC Score:  0.1343

 Confusion Matrix:
[[106551    167]
 [  1364     86]]


In [31]:
# Predict on test data
y_pred = logreg.predict(X_test_nolog_scaled)
y_proba = logreg.predict_proba(X_test_nolog_scaled)[:, 1]
# Evaluate the model
print("Classification Report:\n")
print(classification_report(y_test, y_pred, digits=4))
roc_auc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"PR-AUC Score:  {pr_auc:.4f}")
print("\n Confusion Matrix:")
print(cm)

Classification Report:

              precision    recall  f1-score   support

           0     0.9859    0.9990    0.9924     95415
           1     0.4207    0.0483    0.0867      1428

    accuracy                         0.9850     96843
   macro avg     0.7033    0.5237    0.5396     96843
weighted avg     0.9776    0.9850    0.9791     96843

ROC-AUC Score: 0.8665
PR-AUC Score:  0.1595

 Confusion Matrix:
[[95320    95]
 [ 1359    69]]


### Regularization tuning

In [32]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(logreg, param_grid, scoring='f1', cv=5)
grid.fit(X_train_resampled_nolog_scaled, y_train_resampled.values.ravel())

,estimator,LogisticRegre...ndom_state=42)
,param_grid,"{'C': [0.01, 0.1, ...]}"
,scoring,'f1'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [33]:
print("Best C:", grid.best_params_['C'])
print("Best cross-validated F1 score:", grid.best_score_)

Best C: 10
Best cross-validated F1 score: 0.9893831446019462


In [34]:
# Evaluate on Training Data with Best Model
best_logreg = grid.best_estimator_
y_train_pred_best = best_logreg.predict(X_train_resampled_nolog_scaled)
y_train_proba_best = best_logreg.predict_proba(X_train_resampled_nolog_scaled)[:, 1]
print("Training Classification Report with Best Model:\n")
print(classification_report(y_train_resampled, y_train_pred_best, digits=4))
roc_auc_train_best = roc_auc_score(y_train_resampled, y_train_proba_best)
pr_auc_train_best = average_precision_score(y_train_resampled, y_train_proba_best)
print(f"Training ROC-AUC Score with Best Model: {roc_auc_train_best:.4f}")
print(f"Training PR-AUC Score with Best Model:  {pr_auc_train_best:.4f}")
cm_train_best = confusion_matrix(y_train_resampled, y_train_pred_best)
print("\n Training Confusion Matrix with Best Model:")
print(cm_train_best)

Training Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9835    0.9975    0.9905    786838
           1     0.9975    0.9833    0.9903    786838

    accuracy                         0.9904   1573676
   macro avg     0.9905    0.9904    0.9904   1573676
weighted avg     0.9905    0.9904    0.9904   1573676

Training ROC-AUC Score with Best Model: 0.9983
Training PR-AUC Score with Best Model:  0.9987

 Training Confusion Matrix with Best Model:
[[784881   1957]
 [ 13151 773687]]


In [35]:
# Evaluate the best model on validation set
y_val_pred_best = best_logreg.predict(X_val_nolog_scaled)
y_val_proba_best = best_logreg.predict_proba(X_val_nolog_scaled)[:, 1]
print("Validation Classification Report with Best Model:\n")
print(classification_report(y_val, y_val_pred_best, digits=4))
roc_auc_val_best = roc_auc_score(y_val, y_val_proba_best)
pr_auc_val_best = average_precision_score(y_val, y_val_proba_best)
print(f"Validation ROC-AUC Score with Best Model: {roc_auc_val_best:.4f}")
print(f"Validation PR-AUC Score with Best Model:  {pr_auc_val_best:.4f}")
cm_val_best = confusion_matrix(y_val, y_val_pred_best)
print("\n Validation Confusion Matrix with Best Model:")
print(cm_val_best)

Validation Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9874    0.9984    0.9929    106718
           1     0.3425    0.0600    0.1021      1450

    accuracy                         0.9859    108168
   macro avg     0.6649    0.5292    0.5475    108168
weighted avg     0.9787    0.9859    0.9809    108168

Validation ROC-AUC Score with Best Model: 0.8628
Validation PR-AUC Score with Best Model:  0.1344

 Validation Confusion Matrix with Best Model:
[[106551    167]
 [  1363     87]]


In [36]:
# Evaluate the best model on test set
y_test_pred_best = best_logreg.predict(X_test_nolog_scaled)
y_test_proba_best = best_logreg.predict_proba(X_test_nolog_scaled)[:, 1]
print("Test Classification Report with Best Model:\n")
print(classification_report(y_test, y_test_pred_best, digits=4))
roc_auc_test_best = roc_auc_score(y_test, y_test_proba_best)
pr_auc_test_best = average_precision_score(y_test, y_test_proba_best)
print(f"Test ROC-AUC Score with Best Model: {roc_auc_test_best:.4f}")
print(f"Test PR-AUC Score with Best Model:  {pr_auc_test_best:.4f}")
cm_test_best = confusion_matrix(y_test, y_test_pred_best)
print("\n Test Confusion Matrix with Best Model:")
print(cm_test_best)

Test Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9859    0.9990    0.9924     95415
           1     0.4157    0.0483    0.0866      1428

    accuracy                         0.9850     96843
   macro avg     0.7008    0.5237    0.5395     96843
weighted avg     0.9775    0.9850    0.9791     96843

Test ROC-AUC Score with Best Model: 0.8666
Test PR-AUC Score with Best Model:  0.1595

 Test Confusion Matrix with Best Model:
[[95318    97]
 [ 1359    69]]


## Use original train with class weighting

### Baseline

In [38]:
# Initialize Logistic Regression model
logreg = LogisticRegression(
    max_iter=5000,
    random_state=42,
    class_weight='balanced'
)

# Scale features
scaler = StandardScaler()
X_train_nolog_scaled = scaler.fit_transform(X_train_nolog)
X_val_nolog_scaled = scaler.transform(X_val_nolog)
X_test_nolog_scaled = scaler.transform(X_test_nolog)

In [39]:
# Train (fit) the model
logreg.fit(X_train_nolog_scaled, y_train.values.ravel())

# Evaluate on training data
y_train_pred = logreg.predict(X_train_nolog_scaled)
y_train_proba = logreg.predict_proba(X_train_nolog_scaled)[:, 1]
print("Training Classification Report:\n")
print(classification_report(y_train, y_train_pred, digits=4))
roc_auc_train = roc_auc_score(y_train, y_train_proba)
pr_auc_train = average_precision_score(y_train, y_train_proba)
print(f"Training ROC-AUC Score: {roc_auc_train:.4f}")
print(f"Training PR-AUC Score:  {pr_auc_train:.4f}")
cm_train = confusion_matrix(y_train, y_train_pred)
print("\n Training Confusion Matrix:")
print(cm_train)

Training Classification Report:

              precision    recall  f1-score   support

           0     0.9973    0.8041    0.8903    786838
           1     0.0401    0.7894    0.0763      8151

    accuracy                         0.8040    794989
   macro avg     0.5187    0.7967    0.4833    794989
weighted avg     0.9875    0.8040    0.8820    794989

Training ROC-AUC Score: 0.8756
Training PR-AUC Score:  0.1258

 Training Confusion Matrix:
[[632702 154136]
 [  1717   6434]]


In [40]:
# Predict on validation data
y_pred = logreg.predict(X_val_nolog_scaled)
y_proba = logreg.predict_proba(X_val_nolog_scaled)[:, 1]

# Evaluate the model
print("Classification Report:\n")
print(classification_report(y_val, y_pred, digits=4))

roc_auc = roc_auc_score(y_val, y_proba)
pr_auc = average_precision_score(y_val, y_proba)
cm = confusion_matrix(y_val, y_pred)

print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"PR-AUC Score:  {pr_auc:.4f}")
print("\n Confusion Matrix:")
print(cm)

Classification Report:

              precision    recall  f1-score   support

           0     0.9962    0.8253    0.9027    106718
           1     0.0564    0.7690    0.1051      1450

    accuracy                         0.8245    108168
   macro avg     0.5263    0.7971    0.5039    108168
weighted avg     0.9836    0.8245    0.8920    108168

ROC-AUC Score: 0.8776
PR-AUC Score:  0.1515

 Confusion Matrix:
[[88070 18648]
 [  335  1115]]


In [41]:
# Predict on test data
y_pred = logreg.predict(X_test_nolog_scaled)
y_proba = logreg.predict_proba(X_test_nolog_scaled)[:, 1]
# Evaluate the model
print("Classification Report:\n")
print(classification_report(y_test, y_pred, digits=4))
roc_auc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"PR-AUC Score:  {pr_auc:.4f}")
print("\n Confusion Matrix:")
print(cm)

Classification Report:

              precision    recall  f1-score   support

           0     0.9956    0.8595    0.9225     95415
           1     0.0734    0.7437    0.1336      1428

    accuracy                         0.8577     96843
   macro avg     0.5345    0.8016    0.5280     96843
weighted avg     0.9820    0.8577    0.9109     96843

ROC-AUC Score: 0.8814
PR-AUC Score:  0.1764

 Confusion Matrix:
[[82005 13410]
 [  366  1062]]


### Regularization Tuning

In [42]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(logreg, param_grid, scoring='f1', cv=5)
grid.fit(X_train_nolog_scaled, y_train.values.ravel())

,estimator,LogisticRegre...ndom_state=42)
,param_grid,"{'C': [0.01, 0.1, ...]}"
,scoring,'f1'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [43]:
print("Best C:", grid.best_params_['C'])
print("Best cross-validated F1 score:", grid.best_score_)

Best C: 0.01
Best cross-validated F1 score: 0.07977946999773596


In [44]:
# Evaluate on Training Data with Best Model
best_logreg = grid.best_estimator_
y_train_pred_best = best_logreg.predict(X_train_nolog_scaled)
y_train_proba_best = best_logreg.predict_proba(X_train_nolog_scaled)[:, 1]
print("Training Classification Report with Best Model:\n")
print(classification_report(y_train, y_train_pred_best, digits=4))
roc_auc_train_best = roc_auc_score(y_train, y_train_proba_best)
pr_auc_train_best = average_precision_score(y_train, y_train_proba_best)
print(f"Training ROC-AUC Score with Best Model: {roc_auc_train_best:.4f}")
print(f"Training PR-AUC Score with Best Model:  {pr_auc_train_best:.4f}")
cm_train_best = confusion_matrix(y_train, y_train_pred_best)
print("\n Training Confusion Matrix with Best Model:")
print(cm_train_best)

Training Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9973    0.8041    0.8903    786838
           1     0.0401    0.7901    0.0763      8151

    accuracy                         0.8039    794989
   macro avg     0.5187    0.7971    0.4833    794989
weighted avg     0.9875    0.8039    0.8820    794989

Training ROC-AUC Score with Best Model: 0.8756
Training PR-AUC Score with Best Model:  0.1258

 Training Confusion Matrix with Best Model:
[[632691 154147]
 [  1711   6440]]


In [45]:
# Evaluate the best model on validation set
y_val_pred_best = best_logreg.predict(X_val_nolog_scaled)
y_val_proba_best = best_logreg.predict_proba(X_val_nolog_scaled)[:, 1]
print("Validation Classification Report with Best Model:\n")
print(classification_report(y_val, y_val_pred_best, digits=4))
roc_auc_val_best = roc_auc_score(y_val, y_val_proba_best)
pr_auc_val_best = average_precision_score(y_val, y_val_proba_best)
print(f"Validation ROC-AUC Score with Best Model: {roc_auc_val_best:.4f}")
print(f"Validation PR-AUC Score with Best Model:  {pr_auc_val_best:.4f}")
cm_val_best = confusion_matrix(y_val, y_val_pred_best)
print("\n Validation Confusion Matrix with Best Model:")
print(cm_val_best)

Validation Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9962    0.8251    0.9026    106718
           1     0.0563    0.7676    0.1049      1450

    accuracy                         0.8243    108168
   macro avg     0.5262    0.7963    0.5037    108168
weighted avg     0.9836    0.8243    0.8919    108168

Validation ROC-AUC Score with Best Model: 0.8776
Validation PR-AUC Score with Best Model:  0.1515

 Validation Confusion Matrix with Best Model:
[[88053 18665]
 [  337  1113]]


In [46]:
# Evaluate the best model on test set
y_test_pred_best = best_logreg.predict(X_test_nolog_scaled)
y_test_proba_best = best_logreg.predict_proba(X_test_nolog_scaled)[:, 1]
print("Test Classification Report with Best Model:\n")
print(classification_report(y_test, y_test_pred_best, digits=4))
roc_auc_test_best = roc_auc_score(y_test, y_test_proba_best)
pr_auc_test_best = average_precision_score(y_test, y_test_proba_best)
print(f"Test ROC-AUC Score with Best Model: {roc_auc_test_best:.4f}")
print(f"Test PR-AUC Score with Best Model:  {pr_auc_test_best:.4f}")
cm_test_best = confusion_matrix(y_test, y_test_pred_best)
print("\n Test Confusion Matrix with Best Model:")
print(cm_test_best)

Test Classification Report with Best Model:

              precision    recall  f1-score   support

           0     0.9956    0.8600    0.9228     95415
           1     0.0736    0.7437    0.1340      1428

    accuracy                         0.8583     96843
   macro avg     0.5346    0.8018    0.5284     96843
weighted avg     0.9820    0.8583    0.9112     96843

Test ROC-AUC Score with Best Model: 0.8817
Test PR-AUC Score with Best Model:  0.1773

 Test Confusion Matrix with Best Model:
[[82057 13358]
 [  366  1062]]
